# Super resumen comandos de este notebook

| Comando | descripcion |
|:---|:---|
| ``from langchain.prompts import ChatPromptTemplate`` | Importamos clase para crear plantilla de prompt |
| ``template_string = 'hazme un resumen del libro {titulo_libro}'`` | Creamos una string base para el prompt donde lo que esta entre llaves indica variables del prompt |
| ``prompt_template = ChatPromptTemplate.from_template(template_string)`` | Creamos una plantilla base a partir del anterior  |
| ``template_filled = prompt_template.format_messages(titulo_libro="Harry potter")`` | Instanciamos una plantilla de prompt con determinados valores |
| ``from langchain.chat_models import ChatOpenAI`` | Creamos nuestro modelo LLM de tipo chat, en este caso usaremos el de openAI |
|`` chat(template_filled)``| Invocamos al modelo de chat insertandole el prompt |
|``from langchain.output_parsers import ResponseSchema``| Importamos la clase que nos sirve para esquematizar la salida|
|``from langchain.output_parsers import ResponseSchema``| Importamos la clase que nos sirve para esquematizar la salida|

## Conceptos previos
Un LLM es un tipo de inteligencia artificial que puede generar y comprender texto humano. Los LLMs se entrenan en enormes conjuntos de datos de texto y código, lo que les permite aprender las reglas del lenguaje y cómo usarlas para generar texto de forma significativa.

**Modelos**: se refieren a los modelos de lenguaje grande (LLM) que sustentan gran parte de su funcionamiento

**Los prompts** son instrucciones que se dan a un LLM para indicarle qué tipo de texto debe generar. Un prompt puede ser tan simple como una frase o tan complejo como un párrafo entero. Por ejemplo, podrías darle a un LLM el prompt "Escribe un poema sobre un gato" o "Escribe un artículo de noticias sobre la última investigación sobre el cambio climático".

**Los parsers**: se utilizan para analizar la salida de un LLM y convertirla en un formato más estructurado. Por ejemplo, un parser podría tomar la salida de un LLM que es un poema y dividirlo en estrofas y versos. O un parser podría tomar la salida de un LLM que es un artículo de noticias y extraer los datos clave, como el titular, la fecha y el autor.

normalmente cuando se entrena un modelo LLM se cumple el mismo patrón de hacer un promt, ejecutar inferencia del modelo de LLM y luego parsear la respuesta del modelo, de está manera nacio **LangChain** que intenta crear un entorno de trabajo que permita abstraer todo el proceso y simplificarlo.

## LangChain

Tal y como se comento anteriormente LangChain es un marco de trabajo que facilita la construcción de aplicaciones que puedan explotar los LLMs proporcionando una serie de componentes que se pueden combinar para crear cadenas de operaciones complejas. Una cadena típica podría incluir un paso para solicitar un modelo, un paso para analizar la salida y un paso para guardar los resultados en una base de datos.

| Componente | Descripción |
|------------|-------------|
| Modelos    | LangChain proporciona una biblioteca de modelos LLM preentrenados que se pueden utilizar para una variedad de tareas. |
| Formatters | Los formatters se utilizan para parsear la salida de los modelos LLM en un formato más estructurado. |
| Pipelines  | Las pipelines son secuencias de componentes que se pueden combinar para crear cadenas de operaciones complejas. |

## Instalacion de lang chain

In [10]:
!pip install --upgrade langchain

Nombre_Paquete,Versión
langchain,0.0.316
pandas,1.5.3


## Seleccion del modelo 
lanchaing ofrece abstracciones de modelos conocidos de tipo LLM (large language model), modelos de chat o text embedding Models, estos ultimos convierten un token, sentencias o texto en vectores numericos que capturan información semántica y relaciones. semanticas. En el siguiente ejemplo se vera como utilizamos un modelo de chat popular, que es el de openai, aunque hay miles de modelos más que se pueden utilizar

In [2]:
from langchain.chat_models import ChatOpenAI
import os

os.environ['OPENAI_API_KEY'] = 'Hay que asignar una key aquí'
llm_model = "gpt-3.5-turbo"
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0

# Internamente buscara en nuestro sistema operativo 
chat = ChatOpenAI(temperature=0.0, model=llm_model)
chat

ChatOpenAI(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, temperature=0.0, openai_api_key='Hay que asignar una key aquí', openai_api_base='', openai_organization='', openai_proxy='')

## Creación de un promp template

En el siguiente segmento de codigo se apreciará como se crea un template base que sirve como prompt para el modelo, es como una especie de plantilla, que no servirá para hacer peticiones similares con diferentes valores segun el caso que se requiera, por ejemplo, si queremos crear una aplicación cuya función principal es hacer resumenes de libros, vemos que el hecho de hacer un resumen es algo común, pero el libro especifico que se desea resumir varía según el titulo de libro que se provea, otro ejemplo es el expuesto a continuación: 

In [11]:
from langchain.prompts import ChatPromptTemplate

# Como apreciamos la manera de decirle a lang chain que 
# es lo que varia segun la ejecucion es utilizando las llaves
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

# creamos una plantilla de prompt a partir de la 
# cadena de texto anterior
prompt_template = ChatPromptTemplate.from_template(template_string)


# Creamos variables para una instancia concreta de la 
# plantilla
customer_style = """American English \
in a calm and respectful tone
"""

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

# Aqui creamos una especia de instancia de la plantilla 
# del prompt utilizando keyword similares a los nombres
# asignados mas arriba entre llaves
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)


# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)

print(customer_response.content)

NameError: name 'chat' is not defined

## Parsear la salida

Una vez tenemos definido nuestro modelo, nuestro prompt template, lo siguiente es definir como queremos que nos llegue la respuesta, el LLM nos dará una salida general y nuestro objetivo es que nos devuelva la información esquematizada o estructurada de una determinada manera para luego nosotros poder trabajar con ella. A continuación se muestra un ejemplo:

In [4]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""
prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

NameError: name 'customer_review' is not defined